In [16]:
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

import pandas as pd

In [17]:
train_image_path = r'C:\Users\DELL\Desktop\DataSet\Image_Dataset\train/'
test_image_path = r'C:\Users\DELL\Desktop\DataSet\Image_Dataset\test/'

In [19]:
img = cv2.imread(train_image_path + "\\" + os.listdir(train_image_path)[0])

In [ ]:
os.listdir(train_image_path)[860:870]

In [24]:
img.shape

(600, 600, 3)

In [25]:
scale_percent = 60 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)

In [26]:
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

In [29]:
train_labels = [file_name.split(" ")[0] for file_name in os.listdir(train_image_path)]
test_labels = [file_name.split(" ")[0] for file_name in os.listdir(test_image_path)]

In [30]:
train_df = pd.DataFrame(columns=["img_name","label"])
train_df["img_name"] = os.listdir(train_image_path)
for idx, i in enumerate(os.listdir(train_image_path)):
    if "bellflower" in i:
        train_df["label"][idx] = 0
    if "daisy" in i:
        train_df["label"][idx] = 1
    if "dandelion" in i:
        train_df["label"][idx] = 2
    if "lotus" in i:
        train_df["label"][idx] = 3    
    if "tulip" in i:
        train_df["label"][idx] = 4
    if "rose" in i:
        train_df["label"][idx] = 5
    if "sunflower" in i:
        train_df["label"][idx] = 6    

In [32]:
train_df.to_csv(r'train_csv.csv', index = False, header=True)

In [33]:
test_df = pd.DataFrame(columns=["img_name","label"])
test_df["img_name"] = os.listdir(test_image_path)
for idx, i in enumerate(os.listdir(test_image_path)):
    if "bellflower" in i:
        train_df["label"][idx] = 0
    if "daisy" in i:
        train_df["label"][idx] = 1
    if "dandelion" in i:
        train_df["label"][idx] = 2
    if "lotus" in i:
        train_df["label"][idx] = 3    
    if "tulip" in i:
        train_df["label"][idx] = 4
    if "rose" in i:
        train_df["label"][idx] = 5
    if "sunflower" in i:
        train_df["label"][idx] = 6  

In [35]:
test_df.to_csv (r'test_csv.csv', index = False, header=True)

# Creating Model

In [36]:
import torch
import torchvision
import torchvision.transforms as transforms


import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

import torch.optim as optim

from tqdm import tqdm_notebook

from PIL import Image

import torchvision.models as models

from torch.utils.data import Dataset

import warnings

In [37]:
class Pistachio(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [38]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [39]:
transform = transforms.Compose(
        [
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [55]:
batch_size = 10
shuffle = True
pin_memory = True

In [56]:
dataset = Pistachio(train_image_path, "train_csv.csv",transform=transform)
train_set, validation_set = torch.utils.data.random_split(dataset,[1500,115])

train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size, pin_memory=pin_memory)

In [57]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [62]:
model

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [63]:
for epoch in tqdm_notebook(range(20)):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device=device))
        loss = criterion(outputs.to(device=device), labels.type(torch.LongTensor).to(device=device))
        loss.backward()
        optimizer.step()

        print(loss)

C:\Users\Admin\AppData\Local\Temp\ipykernel_29428\2179853303.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0

Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`

  for epoch in tqdm_notebook(range(20)):  # loop over the dataset multiple times


  0%|          | 0/20 [00:00<?, ?it/s]

tensor(2.2252, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(2.1695, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(2.0569, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.9731, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.9232, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.7438, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.4547, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(0.9669, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(2.2468, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.6510, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.5802, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.3813, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(0.9749, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(0.7129, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.2692, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(0.9884, device='cuda:0', grad_fn=<NllLossBackward>)

tensor(1.6007, device='cuda:0', grad_fn=

In [65]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in validation_loader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 80 %


In [75]:
classes = ["0", "1"]

In [94]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in validation_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label.cpu() == prediction.cpu():
                correct_pred[classes[int(label.cpu().numpy())]] += 1
            total_pred[classes[int(label.cpu().numpy())]] += 1